In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow_datasets as tfds

In [ ]:
(train_data,test_data),ds_info=tfds.load(name='food101',
                                         split=["train","validation"],
                                         shuffle_files=True,
                                         as_supervised=True,
                                         with_info=True
                                         )

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
def preprocess(image,label,shape=224):
  image=tf.image.resize(image,[shape,shape])
  return tf.cast(image,tf.float32),label

In [ ]:
for image,label in train_data.take(1):
  print(preprocess(image,label.numpy()))

In [ ]:
train_data=train_data.map(map_func=preprocess,num_parallel_calls=tf.data.AUTOTUNE)
train_data=train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data=test_data.map(map_func=preprocess,num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
train_data,test_data

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

d_a=tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
])

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [ ]:

bm=tf.keras.applications.efficientnet_v2.EfficientNetV2B2(include_top=False)
bm.trainable=False
inputs=tf.keras.layers.Input(shape=(224,224,3))
x=d_a(inputs)
x=bm(x)
x=tf.keras.layers.GlobalMaxPool2D()(x)
x=tf.keras.layers.Dense(101)(x)
outputs=tf.keras.layers.Activation("softmax",dtype=tf.float32)(x)

model1=tf.keras.Model(inputs,outputs)

model1.compile(optimizer="adam",loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
hist1=model1.fit(train_data,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=(0.25*len(test_data)),epochs=5)

In [ ]:
model1.predict(test_data)